In [1]:
!pip -q install langchain langchain_community pypdf fastembed chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.9 MB/s eta 0:00:0

In [2]:
import os
from getpass import getpass

hf_token = getpass("Paste your Hugging Face token here: ")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

StdinNotImplementedError: getpass was called, but this frontend does not support input requests.

### verifying your token

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
user = api.whoami(token=os.environ["HUGGINGFACEHUB_API_TOKEN"])
print("Authenticated as:", user.get("name", user.get("user")))


In [1]:
import os

dataset_path = "/Data/"

for root, dirs, files in os.walk(dataset_path):
    for name in files:
        print(os.path.join(root, name))

In [5]:
# import PDF reader
from langchain_community.document_loaders.pdf import PyPDFLoader
loader = PyPDFLoader("Data/stories.pdf")  # Adjust path to match your file location
data = loader.load()

# Verify loading
print(f"Loaded {len(data)} pages")
print("First page snippet:")
print(data[0].page_content[:200]) 

Loaded 11 pages
First page snippet:
1/I SELL MY DREAMS
Short stories
INTRODUCTION
A short story is a prose narrative of limited length.
It organises the action and thoughts of its
characters into the pattern of a plot. The plot
form may


In [6]:
# import text splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
chunk_size = 512,
chunk_overlap = 0,
)

chunks = text_splitter.split_documents(data)


In [ ]:
len(chunks)

In [ ]:
chunks[0]

In [ ]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

model_name = "thenlper/gte-large"
embedding_model = FastEmbedEmbeddings(model_name="thenlper/gte-large")


In [ ]:
from langchain.vectorstores import Chroma

# initialize the vector store (save to disk)
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db")

In [ ]:
# Let's define the query, since we are going to use it multiple times.
query = "what is importance of I sell my dreams according to the author"

In [ ]:
# retrieve from vector db (load from disk) with query
retrieved_docs = db.similarity_search(query)
print(retrieved_docs[0].page_content)


In [ ]:
# initialize the retriever
retriever = db.as_retriever(
search_type="mmr", #similarity
search_kwargs={'k': 4}
)

In [ ]:
!pip install --upgrade langchain-huggingface


In [ ]:
from langchain_huggingface.llms.huggingface_endpoint import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    model="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    temperature=0.1,
    max_new_tokens=512,
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """
<s>[INST]
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT
[/INST]
CONTEXT: {context}
</s>
[INST]
{query}
[/INST]
"""


In [ ]:
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

In [ ]:
chain = (
{"context": retriever, "query": RunnablePassthrough()}
| prompt
| llm
| output_parser
)


In [ ]:
response = chain.invoke(query)
response

In [ ]:
response3 = chain.invoke("what is the author's name")
response3

### **NICE*# *** suuuuuuuuuuuuuuuuuuuuuuuuuuuui